# Height

In [128]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *
import warnings
warnings.filterwarnings("ignore")

In [129]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

database connected


In [130]:
df = pd.read_csv("../Data/reintubation_df_240127.csv", index_col="Unnamed: 0")
df2 = df.copy()
df2

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN


------

In [131]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.d_items;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
d_items = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  0.04700064659118652


## d_items -> heights 데이터 확인

* itemid => 226707, 226730

In [133]:
d_items[d_items["label"].str.contains('Height', case=False)]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1941,226707,Height,Height,chartevents,General,Inch,Numeric,NaN,NaN
1945,226730,Height (cm),Height (cm),chartevents,General,cm,Numeric,NaN,NaN


## chartevents에서 키에 해당하는 itemid 추출

In [91]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.chartevents where itemid in (226707, 226730);")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
heights_chart = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  22.78012704849243


In [92]:
heights_chart

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,88981,2180-07-23 12:36:00,2180-07-23 14:45:00,226707,60,60.0,Inch,0
1,10000032,29079034,39553978,88981,2180-07-23 12:36:00,2180-07-23 14:45:00,226730,152,152.0,cm,0
2,10001725,25563031,31205490,22280,2110-04-11 15:52:00,2110-04-11 16:01:00,226707,62,62.0,Inch,0
3,10001725,25563031,31205490,22280,2110-04-11 15:52:00,2110-04-11 16:01:00,226730,157,157.0,cm,0
4,10001884,26184834,37510196,3512,2131-01-11 04:20:00,2131-01-13 14:43:00,226707,62,62.0,Inch,0
...,...,...,...,...,...,...,...,...,...,...,...
67409,19997843,20277361,32880062,7326,2120-11-17 23:17:00,2120-11-18 02:58:00,226730,157,157.0,cm,0
67410,19998843,24842066,30988867,8549,2187-02-05 09:28:00,2187-02-06 11:21:00,226707,71,71.0,Inch,0
67411,19998843,24842066,30988867,8549,2187-02-05 09:28:00,2187-02-06 11:21:00,226730,180,180.0,cm,0
67412,19999297,21439025,37364566,30382,2162-08-16 05:48:00,2162-08-16 11:48:00,226707,65,65.0,Inch,0


In [137]:
# heights 값이 cm, inch가 각각 환자당 1개씩 저장되어 있는 것을 확인

heights_chart['valueuom'].value_counts()

valueuom
Inch    33707
cm      33707
Name: count, dtype: int64

In [138]:
# cm 값만 사용

heights_chart2 = heights_chart[heights_chart["valueuom"] == "cm"].copy()

In [139]:
heights_chart2

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
1,10000032,29079034,39553978,88981,2180-07-23 12:36:00,2180-07-23 14:45:00,226730,152,152.0,cm,0
3,10001725,25563031,31205490,22280,2110-04-11 15:52:00,2110-04-11 16:01:00,226730,157,157.0,cm,0
5,10001884,26184834,37510196,3512,2131-01-11 04:20:00,2131-01-13 14:43:00,226730,157,157.0,cm,0
7,10002013,23581541,39060235,96730,2160-05-18 10:00:00,2160-05-18 12:05:00,226730,157,157.0,cm,0
9,10002428,20321825,34807493,62692,2156-04-30 20:36:00,2156-05-02 16:37:00,226730,150,150.0,cm,0
...,...,...,...,...,...,...,...,...,...,...,...
67405,19995127,27369164,32899958,86355,2138-06-07 07:14:00,2138-06-07 07:15:00,226730,185,185.0,cm,0
67407,19997752,29452285,34531437,33390,2128-03-02 11:09:00,2128-03-06 19:55:00,226730,170,170.0,cm,0
67409,19997843,20277361,32880062,7326,2120-11-17 23:17:00,2120-11-18 02:58:00,226730,157,157.0,cm,0
67411,19998843,24842066,30988867,8549,2187-02-05 09:28:00,2187-02-06 11:21:00,226730,180,180.0,cm,0


In [140]:
len(heights_chart2)

33707

In [141]:
df[df["subject_id"] == 10004401]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN


In [142]:
heights_chart2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33707 entries, 1 to 67413
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   subject_id    33707 non-null  int64         
 1   hadm_id       33707 non-null  int64         
 2   stay_id       33707 non-null  int64         
 3   caregiver_id  33707 non-null  int64         
 4   charttime     33707 non-null  datetime64[ns]
 5   storetime     33707 non-null  datetime64[ns]
 6   itemid        33707 non-null  int64         
 7   value         33707 non-null  object        
 8   valuenum      33707 non-null  float64       
 9   valueuom      33707 non-null  object        
 10  warning       33707 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(6), object(2)
memory usage: 3.1+ MB


In [127]:
'''
heights 값 같은 경우는 변동이 거의 없다고 인지했지만
chart를 보니 stay_id, charttime 등 시간대로 나뉘어져 있는 것으로 확인
일관적으로 넣는 것보단 이전 방법처럼 int_time, ext_time 기준으로 병합
'''


def group_df_heights(group):
    for idx, row in group.iterrows():
        if pd.notnull(row["int_stayid"]):
            
            height_df = heights_chart2[(heights_chart2["subject_id"] == row["subject_id"]) & (heights_chart2["hadm_id"] == row["hadm_id"]) & (heights_chart2["stay_id"] == row["int_stayid"])]
            suitable_location = height_df[height_df["charttime"] <= row["intubationtime"]].last_valid_index()
            
            try:
                group.loc[idx, "heights"] = height_df.at[suitable_location, "valuenum"]
            except:
                try:
                    group.loc[idx, "heights"] = height_df.at[height_df["charttime"].idxmin(), "valuenum"]
                except:
                    group.loc[idx, "heights_marker"] = "suitable_location is null, about int_stayid"

        else:
            
            height_df = heights_chart2[(heights_chart2["subject_id"] == row["subject_id"]) & (heights_chart2["hadm_id"] == row["hadm_id"]) & (heights_chart2["stay_id"] == row["ext_stayid"])]
            suitable_location = height_df[height_df["charttime"] >= row["extubationtime"]].first_valid_index()
            
            try:
                group.loc[idx, "heights"] = height_df.at[suitable_location, "valuenum"]
            except:
                try:
                    group.loc[idx, "heights"] = height_df.at[height_df["charttime"].idxmax(), "valuenum"]
                except:
                    group.loc[idx, "heights_marker"] = "suitable_location is null, about ext_stayid"
                    
    return group

In [109]:
df_heights = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_heights)

In [144]:
df_heights[df_heights["heights"].isna()]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,heights,heights_marker
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
8,10015931,22130791,37093652.0,2177-03-24 21:47:00,2177-03-25 09:57:00,224385.0,59.5,37093652.0,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,2177-03-29 14:15:00,2177-03-29 14:15:00,NaN,5954.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
12,10020740,23831430,NaN,2150-03-11 15:34:00,NaN,NaN,NaN,35026312.0,2150-03-12 18:15:00,227194.0,102.5,Planned Extubation,2150-04-25 13:50:00,NaN,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about ext_stayid"
13,10020740,23831430,NaN,2150-03-11 15:34:00,NaN,NaN,NaN,35044342.0,2150-03-19 11:05:00,227194.0,103.0,Planned Extubation,2150-04-25 13:50:00,NaN,NaN,NaN,2150-03-27 07:44:00,11319.0,NaN,"suitable_location is null, about ext_stayid"
15,10020740,23831430,31077365.0,2150-03-11 15:34:00,2150-03-30 08:20:00,224385.0,103.0,31077365.0,2150-04-02 01:09:00,227194.0,103.0,Planned Extubation,2150-04-25 13:50:00,NaN,NaN,3889.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8978,19969918,29544887,31213286.0,2186-01-15 23:00:00,2186-01-16 21:10:00,224385.0,68.0,31213286.0,2186-01-18 09:56:00,227194.0,68.0,Planned Extubation,2186-02-07 17:04:00,NaN,NaN,2206.0,2186-01-27 18:30:00,13474.0,NaN,"suitable_location is null, about int_stayid"
8983,19971748,21221864,35213788.0,2178-01-29 05:48:00,2178-01-29 09:00:00,224385.0,72.1,35213788.0,2178-01-29 13:12:00,227194.0,72.1,Planned Extubation,2178-02-13 16:15:00,NaN,NaN,252.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
8988,19975827,26216274,30234904.0,2180-03-22 12:33:00,2180-03-22 12:45:00,224385.0,121.0,30234904.0,2180-03-23 10:00:00,227194.0,121.0,Planned Extubation,2180-03-23 19:30:00,NaN,NaN,1275.0,NaN,NaN,NaN,"suitable_location is null, about int_stayid"
8989,19979469,29434840,37363141.0,2202-02-22 18:26:00,2202-02-22 21:40:00,224385.0,86.0,NaN,NaN,NaN,NaN,NaN,2202-02-28 18:20:00,NaN,NaN,NaN,NaN,NaN,NaN,"suitable_location is null, about int_stayid"


# 내용정리

* d_items 에서 Heights에 관련된 itemid 확인, chartevents에서 해당되는 데이터만 추출
* cm, inch 값이 데이터 크기가 동일, cm 데이터만 활용
* heights를 paired된 data에 병합

## 문의사항

* NaN 값을 대체하는 방법..?
    * 함수에서 기본적으로 subject_id, hadm_id, stay_id(int_stayid or ext_stayid) 값이 동일한 값만 병합이 되었습니다.
    * 이 조건 때문에 nan값이 발생하는 것 같은데, 키 같은 경우는 변동이 거의 없는 편이니 subject_id 기준으로 left_join으로 진행하는 것도 괜찮을까요?